In [1]:
import networkx as nx
from pyvis.network import Network
import pandas as pd
import math
import matplotlib.pyplot as plt

In [24]:
df = pd.read_excel('raw_data/cartel_shareholders/Aalberts.xlsx')

# drop last 3 rows
df = df[:-3]

df_cleaned = df.dropna(how='all').dropna(axis=1, how='all')

# Get relevant columns with % O/S in the name
relevant_columns_percent = df.columns[df.columns.str.contains('% O/S')]
relevant_columns_filing = df.columns[df.columns.str.contains('Filing Type')]

df_cleaned = df_cleaned[['Investor Name', 'Investor Sub-Type'] + list(relevant_columns_percent) + list(relevant_columns_filing)]

# Change column names to be more readable by renaming % O/S to perc_os_{month_year} and Filing Type to filing_type_{month_year}
col_list = []

for col in df_cleaned.columns:
    if '% O/S' in col:
        month_year = col.split(' ')[-1]
        col_list.append(f'perc_os_{month_year}')
    elif 'Filing Type' in col:
        month_year = col.split(' ')[-1]
        col_list.append(f'filing_type_{month_year}')
    else:
        col_list.append(str.lower(str.replace(col, ' ', '_')))

df_cleaned.columns = col_list

df_cleaned.head()

,investor_name,investor_sub-type,perc_os_31-Dec-2011,perc_os_31-Dec-2010,perc_os_31-Dec-2009,perc_os_31-Dec-2008,perc_os_31-Dec-2007,perc_os_31-Dec-2006,perc_os_31-Dec-2005,perc_os_31-Dec-2004,...,filing_type_31-Dec-2006,filing_type_31-Dec-2005,filing_type_31-Dec-2004,filing_type_31-Dec-2003,filing_type_31-Dec-2002,filing_type_31-Dec-2001,filing_type_31-Dec-2000,filing_type_31-Dec-1999,filing_type_31-Dec-1998,filing_type_31-Dec-1997
0,Étoile Gestion,Investment Advisor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,...,NaN,NaN,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WestLB Mellon Asset Management Kapitalanlagege...,Investment Advisor,0.03,0.05,0.03,0.14,0.17,0.05,0.06,0.07,...,Aggregate MFs,Aggregate MFs,Aggregate MFs,NaN,NaN,Aggregate MFs,Aggregate MFs,NaN,Aggregate MFs,Aggregate MFs
2,Wealth Management Partners NV,Investment Advisor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Other Substantial/Declarable,Other Substantial/Declarable,NaN,NaN,NaN
3,Waddell & Reed Investment Management Company,Investment Advisor,NaN,NaN,NaN,0.00,0.22,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,W & W Asset Management GmbH,Investment Advisor,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Get all Files in raw_data/cartel_shareholders folder
import os
files = os.listdir("raw_data/cartel_shareholders")
files = [f for f in files if f.endswith(".xlsx")]

# Create a list of all the dataframes
dfs = []
for f in files:
    df = pd.read_excel("raw_data/cartel_shareholders/" + f)
    
        # drop last 3 rows
    df = df[:-3]

    df_cleaned = df.dropna(how='all').dropna(axis=1, how='all')

    # Get relevant columns with % O/S in the name
    relevant_columns_percent = df.columns[df.columns.str.contains('% O/S')]
    relevant_columns_filing = df.columns[df.columns.str.contains('Filing Type')]

    df_cleaned = df_cleaned[['Investor Name', 'Investor Sub-Type'] + list(relevant_columns_percent) + list(relevant_columns_filing)]

    # Change column names to be more readable by renaming % O/S to perc_os_{month_year} and Filing Type to filing_type_{month_year}
    col_list = []

    for col in df_cleaned.columns:
        if '% O/S' in col:
            month_year = col.split(' ')[-1]
            col_list.append(f'perc_os_{month_year}')
        elif 'Filing Type' in col:
            month_year = col.split(' ')[-1]
            col_list.append(f'filing_type_{month_year}')
        else:
            col_list.append(str.lower(str.replace(col, ' ', '_')))

    df_cleaned.columns = col_list

    df_cleaned[""] = f.split(".")[0]
    dfs.append(df_cleaned)

print(dfs)

In [8]:
# Get all Files in raw_data/cartel_shareholders folder
import os
files = os.listdir("raw_data/cartel_shareholders")
files = [f for f in files if f.endswith(".xlsx")]

# Create a list of all the dataframes
dfs = []
for f in files:
    df = pd.read_excel("raw_data/cartel_shareholders/" + f)
    df_cleaned = df.dropna(how="all").dropna(axis=1, how="all")
    df_cleaned["Cartel Name"] = f.split(".")[0]
    dfs.append(df_cleaned)

print(dfs)

[    % O/S - AALB-AE 31-Dec-2011  % O/S - AALB-AE 31-Dec-2010  \
0                           NaN                          NaN   
1                          0.03                         0.05   
2                           NaN                          NaN   
3                           NaN                          NaN   
4                             0                          NaN   
..                          ...                          ...   
276                         NaN                          NaN   
277                         NaN                          NaN   
278                         NaN                          NaN   
279                       Total                        54.13   
281   Source: Thomson Financial                          NaN   

     % O/S - AALB-AE 31-Dec-2009  % O/S - AALB-AE 31-Dec-2008  \
0                            NaN                          NaN   
1                           0.03                         0.14   
2                            NaN   